## Status de Jobs

In [0]:
%sql
WITH resultados AS (
  SELECT 
    CASE 
      WHEN result_state = 'SUCCEEDED' THEN 'SUCCESS'
      WHEN result_state IN ('ERROR', 'FAILED', 'SKIPPED') THEN 'ERROR'
      WHEN result_state = 'TIMED_OUT' THEN 'TIMED_OUT'
      ELSE 'OTHER'
    END AS result_state
  FROM system.lakeflow.job_task_run_timeline
  WHERE period_start_time > (SELECT MIN(change_time) FROM system.lakeflow.jobs)
    AND job_id IN (
      SELECT DISTINCT job_id
      FROM system.lakeflow.jobs
      WHERE change_time > (SELECT MAX(delete_time) + INTERVAL 1 MINUTE FROM system.lakeflow.jobs)
    )
)
SELECT 
  result_state,
  COUNT(*) AS total_count
FROM resultados
GROUP BY result_state
ORDER BY total_count DESC;


## Task com mais Erros

In [0]:
%sql
SELECT 
  task_key,
  COUNT(*) AS error_count
FROM system.lakeflow.job_task_run_timeline
WHERE result_state IN ('ERROR', 'FAILED', 'SKIPPED')
  AND period_start_time > (SELECT MIN(change_time) FROM system.lakeflow.jobs)
  AND job_id IN (
    SELECT DISTINCT job_id
    FROM system.lakeflow.jobs
    WHERE change_time > (SELECT MAX(delete_time) + INTERVAL 1 MINUTE FROM system.lakeflow.jobs)
  )
GROUP BY task_key
ORDER BY error_count DESC

## Lista de Task com maiores duração